## Read image data

In [28]:
import os
import pprint
pp = pprint.PrettyPrinter()


""" Move into comfy.py """
from PIL import Image
import json


def raw_image_info(filename: str) -> dict[str, str]:
    img = Image.open(filename)
    if filename[-4:] == ".png":
        info = img.load()
        return img.info
    # This should just return exif_data and parse elsewhere
    elif filename[-4:] == ".jpg":
        exif_data = img._getexif()
        return exif_data[37510].decode('utf-8').replace('\x00', '').replace('UNICODE', '')

def comfy_prompt(image_info):
    return json.loads(image_info['prompt'])

# If you need to add to this list, find new node types here. [print(nt) for nt in comfy_unique_node_types(FILE_PATH)]
PROMPT_NODE_TYPES = {'DPCombinatorialGenerator': 'text', 'CLIPTextEncode': 'text', 'Power Prompt (rgthree)': 'prompt'}
def scrape_node(node):
    class_type = node['class_type']
    if class_type in PROMPT_NODE_TYPES:
        return node['inputs'][PROMPT_NODE_TYPES[class_type]]
    else:
        raise Exception('Cannot parse node: ' + str(node))

class ComfyImage:
    def __init__(self, filename: str, basepath = ''):
        self.filename = filename
        self.prompt = comfy_prompt(raw_image_info(basepath + filename))
    def nodes(self):
        return {id: value for (id, value) in self.prompt.items()}
    def unique_nodes(self):
        node_types = [value['class_type'] for (_, value) in self.prompt.items()]
        return sorted(set(node_types))
    def text_nodes(self):
        return [value for (_, value) in self.prompt if value['class_type'] in PROMPT_NODE_TYPES]
    def text_values(self):
        return map(scrape_node, self.text_nodes())



In [66]:

# TODO: Dump all info

BASE_PATH = os.path.expanduser('~/ComfyUI/output/damnPonyxlRealistic_damnV20EXTREME/')

images = [img for img in os.listdir(BASE_PATH) if img[-4:] == '.png']

print(f"Processing {len(images)} images")
ImageObjects = [ComfyImage(img, basepath=BASE_PATH) for img in images]
# 34.6s for 1300 images


In [67]:
# LoRA Regex, matches strings as follows
# <lora:TeradaOchiko.safetensors:1.0>
# <lora:Personal/Hex/HexXL_v5_FewerYellow.safetensors:0.5>
import re
from collections import Counter
loraMatch = re.compile(r'<lora:(.*?):([0-9\.]+)>')


loraCounter = Counter()

for img in ImageObjects:
    item340 = img.prompt.get('340', None)
    if item340 is None:
        print("Unable to find node 340 in " + img.filename)
        continue
    if item340['class_type'] != 'Power Prompt (rgthree)':
        print("Unexpected node type: ")
        print(img.filename)
    pprompt = item340['inputs']['prompt']
    # Match Loras
    loras = loraMatch.findall(pprompt)
    for l in loras:
        lora_name = l[0].replace("\\", "/").split('/')[-1].replace('.safetensors', '')
        loraCounter[lora_name] += 1


In [70]:

# Build orderedDictionary from loraCounter Counter object()
loraCounterOut = dict(loraCounter.most_common())

with open('data/loraCounter_damn.json', 'w') as f:
    # Output json dictionary by top counts
    json.dump(loraCounterOut, f, indent=4)

## Analyze data

In [4]:

import os
import pprint
pp = pprint.PrettyPrinter()
import json
from collections import Counter

loraCounter = Counter()

for path in os.listdir('data'):
    if path[-5:] == '.json':
        with open('data/' + path) as f:
            data = json.load(f)
            for lora, value in data.items():
                loraCounter[lora] += value


In [ ]:
loraCounter

In [12]:

targets = {
    'Sirius': [['Sirius', 'bl4ckdr4g'], {}],
    'Bats': [['Bats', 'Aerys', 'TanBat'], {}],
    'Hex': [['Hex'], {}],
    'PixelSketcher': [['PixelSketcher'], {}],
    'Valziel': [['Valziel'], {}],
    'Zaush': [['Zaush'], {}],
    'Tricksta': [['Tricksta'], {}],
}



for name, matchData in targets.items():
    for loraName, count in loraCounter.items():
        if any([loraName.find(match) > -1 for match in matchData[0]]):
            matchData[1][loraName] = count
    matchData[1] = dict(sorted(matchData[1].items(), key = lambda item: item[1], reverse=True))






In [ ]:
targets